In [1]:
import pickle as pkl
import pandas as pd
import io
import uvicorn
import numpy as np
import nest_asyncio
from enum import Enum
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
from fastapi.encoders import jsonable_encoder

In [2]:
#Load Model
similarity = pkl.load(open('model.sav', 'rb'))

In [3]:
#Load Data
def prep_data():
    movies = pd.read_csv("movies_metadata.csv", usecols = [5,9,20], nrows = 5000)
    movies['index'] = [i for i in range(0, len(movies))]
    movies = movies.dropna()
    return movies

movies = prep_data()

In [5]:
#Helper Functions

#Get title of movie
def get_title(index):
    return movies[movies.index == index]["title"].values[0]

#Get index of movie
def get_index(title):
    return movies[movies.title == title]["index"].values[0]

def recommend(user_movie):
    recommendations = sorted(list(enumerate(similarity[get_index(user_movie)])), key = lambda x:x[1], reverse = True)
    #print("The top 3 recommendations for" + " " + user_movie + " " + "are: ")
    #print(get_title(recommendations[1][0]), get_title(recommendations[2][0]), get_title(recommendations[3][0]), sep = "\n")
    top_recos = {'First': get_title(recommendations[1][0]), 'Second': get_title(recommendations[2][0]), 'Third': get_title(recommendations[3][0])}
    return top_recos

In [27]:
#Server

useMyClient = False  #If true both client.ipynb and localhost:8000/docs will work (but data will need to be passed as a json). If false only the docs link will work and you can just enter a string
app = FastAPI(title='Movie Recommender API')

@app.get("/")
def home():
    return "API up and running! Use this link to submit a request http://localhost:8000/docs. OR run client.ipynb"



if(useMyClient):
    @app.post("/predict") 
    def prediction(movie: dict):

        recos = jsonable_encoder(recommend(movie['movie']))

        return JSONResponse(content = recos)
    
else:
    @app.post("/predict") 
    def prediction(movie):

        recos = jsonable_encoder(recommend(movie))

        return JSONResponse(content = recos)

# Allows the server to be run in this interactive environment
nest_asyncio.apply()

# Host depends on the setup you selected (docker or virtual env)
host = "127.0.0.1"

# Spin up the server!    
uvicorn.run(app, host=host, port=8000)

INFO:     Started server process [25836]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:53813 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53813 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:53835 - "POST /predict?movie=Toy%20Story HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [25836]
